In [142]:
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
from skimage.transform import rotate
from skimage.feature import canny
from skimage.io import imread
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from scipy.stats import mode
import cv2
import os

In [143]:
# image1 = cv2.imread(r'E:\test\misaligned_images\5.jpg')
# newImage = image1.copy()

img_path=r'E:\test\misaligned_images'
input_img=sorted([os.path.join(img_path,i) for i in os.listdir(img_path)])
for i in input_img:
    image_org = imread(i)
    image=rgb2gray(image_org)
    
    new_angle=skew_angle_hough_transform(image)
#     print(i,new_angle)
    if new_angle<-45 and new_angle>=-90:
        new_angle+=180
    elif new_angle==0:
        new_angle+=180
    
    print(f"file name: {i}\nangle : {new_angle}")
    rot_img=rotate_image(image_org, new_angle)
    cv2.imwrite(i.replace('misaligned_images','output'),rot_img)

file name: E:\test\misaligned_images\0e27f636242845138158f132878a3351.jpg
angle : 2
file name: E:\test\misaligned_images\0f97914844b786b0d2ef21339286cdf6 - 2.jpg
angle : -2
file name: E:\test\misaligned_images\10.jpg
angle : 92
file name: E:\test\misaligned_images\4.jpg
angle : 1
file name: E:\test\misaligned_images\41.png
angle : 1
file name: E:\test\misaligned_images\5.jpg
angle : -1
file name: E:\test\misaligned_images\5f228310d55c683ed29aa8c0adda7efc.jpg
angle : 90
file name: E:\test\misaligned_images\6 2.jpg
angle : -1
file name: E:\test\misaligned_images\6.jpg
angle : 180
file name: E:\test\misaligned_images\7.jpg
angle : 2
file name: E:\test\misaligned_images\AWT009.jpg
angle : 2
file name: E:\test\misaligned_images\billing-invoice-for-legal-services.jpg
angle : -4
file name: E:\test\misaligned_images\law-firm-invoice-template.jpg
angle : 92


In [131]:
def skew_angle_hough_transform(image):
    # convert to edges
    edges = canny(image)
    # Classic straight-line Hough transform between 0.1 - 180 degrees.
    tested_angles = np.deg2rad(np.arange(0.1, 180.0))
#     tested_angles = np.deg2rad(np.arange(-90.1, 90.0))

    h, theta, d = hough_line(edges, theta=tested_angles)
    
    # find line peaks and angles
    accum, angles, dists = hough_line_peaks(h, theta, d)
    
    # round the angles to 2 decimal places and find the most common angle.
    most_common_angle = mode(np.around(angles, decimals=2))[0]
    
    # convert the angle to degree for rotation.
    skew_angle = np.rad2deg(most_common_angle - np.pi/2)
#     print(skew_angle)
    
#     if skew_angle<-45.0 and skew_angle>-90.00:
#         skew_angle+=90
#     elif skew_angle==0.0:
#         skew_angle+=180
    
#             skew_angle=skew_angle-180

    return int(skew_angle)
    
    
# new_angle=skew_angle_hough_transform(image)    

# rot_img=rotate_image(image_org, new_angle)

# cv2.imwrite(r'E:\test\output\image.jpg',rot_img)

# fig, ax = plt.subplots(ncols=2, figsize=(20,20))
# ax[0].imshow(image, cmap="gray")
# ax[1].imshow(rotate(image, skew_angle_hough_transform(image), cval=1), cmap="gray")

# plt.show()

In [138]:


def rotate_image(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0]) 
    abs_sin = abs(rotation_mat[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat